In [96]:
from dolfin import *
length = 10
width = 10
height = 10

f = Constant((0.0, 0.0, -10))
xdmf = XDMFFile("output/density.xdmf")

# 材料弹性矩阵
E = 210
mu = 0.3
Cons = E/(1+mu)/(1-2*mu)
Cijkl = Cons*as_tensor([[1-mu, mu, mu, 0, 0, 0],
                        [mu, 1-mu, mu, 0, 0, 0],
                        [mu, mu, 1-mu, 0, 0, 0],
                        [1, 0, 0, (1-2*mu)/2, 0, 0],
                        [0, 0, 0, 0, (1-2*mu)/2, 0],
                        [0, 0, 0, 0, 0, (1-2*mu)/2]])

# 应变voigt法表示
def strain3voigt(tensorStrain):
    return as_vector([tensorStrain[0,0],
                      tensorStrain[1,1],
                      tensorStrain[2,2],
                      2*tensorStrain[1,2],
                      2*tensorStrain[2,0],
                      2*tensorStrain[0,1]])

# voigt应力tensor表示
def stress3tensor(voigtVector):
    return as_tensor([[voigtVector[0], voigtVector[5], voigtVector[4]],
                      [voigtVector[5], voigtVector[1], voigtVector[3]],
                      [voigtVector[4], voigtVector[3], voigtVector[2]]])

# 材料应变
def strain(u):
    return 1.0/2.0*(grad(u) + grad(u).T)

# 材料应力
def stress(u):
    return stress3tensor(
        dot(Cijkl, strain3voigt(strain(u)))
        )


In [97]:
# 创建函数空间
# mesh = UnitCubeMesh(1,2,3)
mesh = BoxMesh(Point(0,0,0), Point(length, width, height), length, width, height)
U = VectorFunctionSpace(mesh, "CG", 1)
u = TrialFunction(U)
v = TestFunction(U)

res = inner(stress(u), grad(v)) * dx

In [99]:
# 边界条件和载荷
def clamp_boundary(x, on_boundary):
    return on_boundary and x[1] < DOLFIN_EPS

dirichletBC = DirichletBC(U,
                          Constant([0.0,0.0,0.0]),
                          clamp_boundary)
L = Form(inner(f, v) * dx)
a = Form(inner(stress(u), grad(v)) * dx)
F = dot(v, f) * dx
K = inner(stress(u), grad(v)) * dx

u_solution = Function(U)

solver = LinearVariationalSolver(LinearVariationalProblem(K, F, u_solution, dirichletBC))
xdmf.write(u_solution, 1)